# Quality control

In [16]:
import math
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew
from itertools import groupby
from setting import *

def quality_control(unit_k, df, flags):
    """Quality control for eddy covariance flux data
    Vickers, D., and L. Mahrt, 1997: Quality control and flux sampling problems for tower and aircraft data.
    J. Atmos. Oceanic tech., 14, 512-526
    """
    instrument(df, flags)

    print('%-32s' % '  Diagnostics', end='')
    for var in ['Ux', 'Uy', 'Uz', 'Ts', 'co2', 'h2o']: print('%20s' % var, end='')
    print()

    print('%-32s' % '  Spike fraction', end='')
    for var in ['Ux', 'Uy', 'Uz', 'Ts', 'co2', 'h2o']:
        spikes(df, var, flags)
    print('')

    print('%-32s' % '  Empty bin ratio', end='')
    for var in ['Ux', 'Uy', 'Uz', 'Ts', 'co2', 'h2o']: amplitude_resolution(df, var, flags)
    print()

    print('%-32s' % '  Dropouts, extreme dropouts', end='')
    for var in ['Ux', 'Uy', 'Uz', 'Ts', 'co2', 'h2o']: dropouts(df, var, flags)
    print()

    print('%-32s' % '  Range', end='')
    absolute_limits(df, flags)
    print()

    print('%-32s' % '  Skewness, kurtosis', end='')
    for var in ['Ux', 'Uy', 'Uz', 'Ts', 'co2', 'h2o']:
        detrend(df, var)
        higher_moment_statistics(df, var, flags)
    print()

    print('%-32s' % '  Normalized Harr mean, variance', end='')
    for var in ['Ux', 'Uy', 'Uz', 'Ts', 'co2', 'h2o']: discontinuities(df, var, flags)
    print('\n')

    nonstationary(df, unit_k, flags)
    print()

    first = True
    if any(flags.values()):
        print('  Flags: ', end='')
        for flag in flags:
            if flags[flag] == 1:
                print(f'{flag}' if first else f', {flag}', end='')
                first = False
    print('\n')


def instrument(df, flags):
    """Instrument diagnostics
    If available records for current time period is less than a threshold, or is more than 18000, a flag is placed
    """

    ratio = float(len(df[df['diag'] < 16])) / (AVERAGING_PERIOD_MINUTES * SECONDS_IN_MINUTE * FREQUENCY_HZ)
    print(f'  Available data fraction: {ratio:.3f}\n')
    flags['instrument'] = 1 if (ratio < QC_THRESHOLDS['instrument'] or ratio > 1.0) else 0


def spikes(df, var, flags):
    """Detect and remove spikes
    The method computes the mean and standard deviation for a series of moving windows of length L1. The window moves
    one point at a time through the series. Any point in the window that is more than 3.5 standard deviations from the
    window mean is considered a spike. The point is replaced using linear interpolation between data points. When four
    or more consecutive points are detected, they are not considered spikes and are not replaced. The entire process is
    repeated until no more spikes are detected. During the second pass, when the standard deviations may be smaller if
    spikes were replaced on the previous pass, the threshold for spike detection increases to 3.6 standard deviations
    and a like amount for each subsequent pass. The record is hard flagged when the total number of spikes replaced
    exceeds 1% of the total number of data points.
    """
    CONSECUTIVE_SPIKES = 3
    L1_SECONDS = 300
    std_threshold = 3.5
    window_position = 0
    spike_ind = []
    n_spikes = 0

    L1 = min(int(L1_SECONDS * FREQUENCY_HZ), len(df))

    while True:
        while window_position + L1 <= len(df):
            window_mean = np.nanmean(df.loc[window_position:window_position + L1, var])
            window_std = np.nanstd(df.loc[window_position:window_position + L1, var])

            k0 = window_position + 1
            while k0 < L1:
                if abs(df.loc[k0, var] - window_mean) > std_threshold * window_std:
                    for k in range(k0 + 1, L1):
                        if abs(df.loc[k, var] - window_mean) < std_threshold * window_std: break

                    if (df.loc[k, 'TmStamp'] - df.loc[k0, 'TmStamp']).total_seconds() > CONSECUTIVE_SPIKES / FREQUENCY_HZ:
                        # When four or more consecutive points are detected, they are not considered spikes
                        pass
                    else:
                        # Replace spikes using linear interpolation between data points
                        for kk in range(k0, k):
                            n_spikes += 1
                            if kk not in spike_ind: spike_ind.append(kk)
                            df.loc[kk, var] = np.interp(
                                (df.loc[kk, 'TmStamp'] - df.loc[k0 - 1, 'TmStamp']).total_seconds(),
                                [0, (df.loc[k, 'TmStamp'] - df.loc[k0 - 1, 'TmStamp']).total_seconds()],
                                [df.loc[k0 - 1, var], df.loc[k, var]]
                            )
                    k0 = k
                else:
                    k0 += 1

            window_position += 1

        spike_ratio = float(len(spike_ind)) / float(len(df))
        flags[f'{var}_spikes'] = 1 if (spike_ratio > QC_THRESHOLDS['spike']) else 0

        if n_spikes == 0:
            print('%20.3f' % (spike_ratio), end='')
            return
        else:
            window_position = 0
            n_spikes = 0
            std_threshold += 0.1


def amplitude_resolution(df, var, flags):
    """Detect resolution problems
    A problem is detected by computing a series of discrete frequency distributions for half-overlapping windows of
    length 1000 data points. These windows move one-half the window width at a time through the series. For each window
    position, the number of bins is set to 100 and the interval for the distribution is taken as the smaller of seven
    standard deviations and the range. When the number of empty bins in the discrete frequency distribution exceeds a
    critical threshold value, the record is hard flagged as a resolution problem.
    """

    L1 = 1000
    N_BINS = 100.0

    L1 = min(L1, len(df))
    empty_bins = 0
    window_position = 0

    while (window_position + L1 <= len(df)):
        window_mean = np.nanmean(df.loc[window_position:window_position + L1, var])
        window_std = np.nanstd(df.loc[window_position:window_position + L1, var])
        distribution = min(7 * window_std, np.ptp(df.loc[window_position:window_position + L1, var]))
        bin_edges = np.arange(
            window_mean - distribution / 2.0,
            window_mean + distribution / 2.0 + distribution / N_BINS,
            distribution / N_BINS,
        )

        hist, _ = np.histogram(df.loc[window_position:window_position + L1, var], bins=bin_edges)
        empty_bins = max(empty_bins, float(np.count_nonzero(hist==0)) / N_BINS)

        window_position += 250

    print('%20.3f' %(empty_bins), end='')
    flags[f'{var}_resolution'] = 1 if (empty_bins > QC_THRESHOLDS['empty_bins'])  else 0


def dropouts(df, var, flags):
    """Detect dropouts
    Dropouts are identified using the same window and frequency distributions used for the resolution problem.
    Consecutive points that fall into the same bin of the frequency distribution are tentatively identified as dropouts.
    When the total number of dropouts in the record exceeds a threshold value, the record is flagged for dropouts.
    """
    L1 = 1000
    N_BINS = 100.0

    L1 = min(L1, len(df))
    dropout_ratio = 0
    extreme_dropout_ratio = 0
    window_position = 0

    while (window_position + L1 <= len(df)):
        window_mean = np.nanmean(df.loc[window_position:window_position + L1, var])
        window_std = np.nanstd(df.loc[window_position:window_position + L1, var])
        distribution = min(7 * window_std, np.ptp(df.loc[window_position:window_position + L1, var]))
        bin_edges = np.arange(
            window_mean - distribution / 2.0,
            window_mean + distribution / 2.0 + distribution / N_BINS,
            distribution / N_BINS,
        )

        #https://stackoverflow.com/questions/6352425/whats-the-most-pythonic-way-to-identify-consecutive-duplicates-in-a-list
        bins = np.digitize(df.loc[window_position:window_position + L1, var], bin_edges)
        grouped_bins = [(k, sum(1 for i in g)) for k, g in groupby(bins)]

        max_dropouts = max(grouped_bins, key=lambda x: x[1])

        dropout_ratio = max(dropout_ratio, float(max_dropouts[1]) / float(len(df)))
        if  (max_dropouts[0] < 10 or max_dropouts[0] > 90):
            extreme_dropout_ratio = max(extreme_dropout_ratio, float(max_dropouts[1]) / float(len(df)))

        window_position += 250

    print('%20s' %(f'{dropout_ratio:.3f}, {extreme_dropout_ratio:.3f}'), end='')
    flags[f'{var}_dropouts'] = 1 if (dropout_ratio > QC_THRESHOLDS['dropouts']) else 0
    flags[f'{var}_extreme_dropouts'] = 1 if (extreme_dropout_ratio > QC_THRESHOLDS['extreme_dropouts']) else 0


def absolute_limits(df, flags):
    """Detect unrealistic data out of their physical ranges
    Unrealistic data are detected and hard flagged by simply comparing the minimum and maximum value of all points in
    the record to some fixed limits considered unphysical.
    """
    print('%20s' %(f'[{df["Ux"].min():.3f}, {df["Ux"].max():.3f}]'), end='')
    flags['Ux_absolute_limits'] = 1 if any(abs(df['Ux']) > 30.0) else 0

    print('%20s' %(f'[{df["Uy"].min():.3f}, {df["Uy"].max():.3f}]'), end='')
    flags['Uy_absolute_limits'] = 1 if any(abs(df['Uy']) > 30.0) else 0

    print('%20s' %(f'[{df["Uz"].min():.3f}, {df["Uz"].max():.3f}]'), end='')
    flags['Uz_absolute_limits'] = 1 if any(abs(df['Uz']) > 10.0) else 0

    print('%20s' %(f'[{df["Ts"].min():.2f}, {df["Ts"].max():.2f}]'), end='')
    flags['Ts_absolute_limits'] = 1 if any(df['Ts'] > 60.0) or any(df['Ts'] < -50.0) or np.ptp(df['Ts']) > 10.0 else 0

    print('%20s' %(f'[{df["h2o"].min():.2f}, {df["h2o"].max():.2f}]'), end='')
    flags['h2o_absolute_limits'] = 1 if any(df['h2o'] > 35.0) or any(df['h2o'] < 2.5) or np.ptp(df['h2o']) > 8.0 else 0

    print('%20s' %(f'[{df["co2"].min():.2f}, {df["co2"].max():.2f}]'), end='')
    flags['co2_absolute_limits'] = 1 if any(df['co2'] > 950.0) or any(df['co2'] < 550.0) or np.ptp(df['co2']) > 120.0 else 0


def detrend(df, var):
    """Linear de-trend
    """

    time_array = df['TmStamp'].values.astype(float) / 1.0E9 # Convert to seconds
    time_array -= time_array[0]

    s = np.polyfit(time_array, df[var], 1)

    trend = s[0] * time_array + s[1]
    df[f'{var}_fluct'] = df[var] - trend


def higher_moment_statistics(df, var, flags):
    """Detect possible instrument or recording problems and physical but unusual behavior using higher-moment statistics
    The skewness and kurtosis of the fields are computed for the entire record. The record is hard flagged when the
    skewness is outside the range (-2, 2) or the kurtosis is outside the range (1, 8).
    """
    m3 = skew(df[f'{var}_fluct'])
    m4 = kurtosis(df[f'{var}_fluct'], fisher=False)

    print('%20s' %(f'{m3:.3f}, {m4:.3f}'), end='')

    if abs(m3) > QC_THRESHOLDS['skewness']:
        df[f'{var}_skewness'] = 1

    if abs(m4 - 4.5) > QC_THRESHOLDS['kurtosis']:
        # The range of kurtosis is (1, 8) in Vickers and Mahrt (1997). It is equivalent to having (kurtosis - 4.5) in
        # the range (-3.5, 3.5) in this implementation.
        df[f'{var}_kurtosis'] = 1


def discontinuities(df, var, flags):
    """Detect discontinuities in the data using the Haar transform
    The transform is computed for a series of moving windows of width L1 and then normalized by the smaller of the
    standard deviation for the entire record and one-fourth the range for the entire record. The record is hard flagged
    if the absolute value of any single normalized transform exceeds 3 and soft flagged at 2. To identify coherent
    changes over the window width L1 in the intensity of the fluctuations, we compute the variance for each half-window
    and then compute the difference normalized by the variance over the entire record. The record is hard flagged if the
    absolute value of any single normalized transform exceeds 3 and soft flagged at 2.
    """

    L1_SECONDS = 300
    L1 = min(int(L1_SECONDS * FREQUENCY_HZ), len(df))


    record_std = np.nanstd(df[var])
    record_range = np.ptp(df[var])
    normal = min(record_std, record_range / 4.0)

    window_position = 0

    haar_mean_max = 0
    haar_variance_max = 0

    while (window_position + L1 <= len(df)):
        half_point = int(L1 / 2)

        half1_mean = np.nanmean(df.loc[0:half_point, var])
        half2_mean = np.nanmean(df.loc[half_point:L1, var])

        half1_variance = np.var(df.loc[0:half_point, var])
        half2_variance = np.var(df.loc[half_point:L1, var])

        haar_mean_max = max(haar_mean_max, abs((half2_mean - half1_mean) / normal))
        haar_variance_max = max(haar_variance_max, abs((half2_variance - half1_variance) / (record_std * record_std)))

        window_position += 1

    print('%20s' % (f'{haar_mean_max:.3f}, {haar_variance_max:.3f}'), end='')
    flags[f'{var}_discontinuities'] = 1 if (haar_mean_max > QC_THRESHOLDS['discontinuities'] or haar_variance_max > QC_THRESHOLDS['discontinuities']) else 0


def nonstationary(df, unit_k, flags):
    """Identify nonstaionarity of horizontal wind
    The wind speed reduction is defined as the ratio of the speed of the vector averaged wind to the averaged
    instantaneous speed. When this ratio falls below 0.9, there is some cancellation in the vector average of the wind
    components and a soft flag is raised.
    The alongwind relative nonstationarity is calculated using linear regression to estimate the difference in the
    alongwind component between the beginning and end of the record. This difference normalized by the record mean of
    the alongwind component is used to compute the relative nonstationarity.
    The crosswind relative nonstationarity is computed from the difference based on the regression of the crosswind
    component.
    The flow is classified as nonstationary if RNu, RNv, or RNS > 0.50
    """
    # Wind speed reduction
    vector_average = math.sqrt(df['Ux'].mean() * df['Ux'].mean() + df['Uy'].mean() * df['Uy'].mean())
    instant_average = np.mean(np.sqrt(df['Ux'] * df['Ux'] + df['Uy'] * df['Uy']))
    wind_speed_reduction = vector_average / instant_average

    print(f'  Wind speed reduction: {wind_speed_reduction:.3f}')
    flags['wind_speed_reduction'] = 1 if (wind_speed_reduction < QC_THRESHOLDS['wind_speed_reduction']) else 0

    # Relative nonstationarity
    ## Calculate alongwind and crosswind components
    unit_j = np.cross(unit_k, [df['Ux'].mean(), df['Uy'].mean(), df['Uz'].mean()])
    unit_j /= np.linalg.norm(unit_j)
    unit_i = np.cross(unit_j, unit_k)

    u = df['Ux'] * unit_i[0] + df['Uy'] * unit_i[1] + df['Uz'] * unit_i[2]
    v = df['Ux'] * unit_j[0] + df['Uy'] * unit_j[1] + df['Uz'] * unit_j[2]

    time_array = df['TmStamp'].values.astype(float) / 1.0E9 # Convert to seconds
    time_array -= time_array[0]
    su = np.polyfit(time_array, u, 1)
    du = su[0] * (time_array[-1] - time_array[0])
    sv = np.polyfit(time_array, v, 1)
    dv = sv[0] * (time_array[-1] - time_array[0])

    rnu = du / np.mean(u)
    rnv = dv / np.mean(u)
    rns = math.sqrt(du * du + dv * dv) / np.mean(u)

    print(f'  Alongwind relative nonstationarity: {rnu:.3f}')
    print(f'  Crosswind relative nonstationarity: {rnv:.3f}')
    print(f'  Relative nonstationarity: {rns:.3f}')

    if (rnu > QC_THRESHOLDS['relative_nonstationarity'] or
        rnv > QC_THRESHOLDS['relative_nonstationarity'] or
        rns > QC_THRESHOLDS['relative_nonstationarity']):
        flags['relative_nonstationarity'] = 1

# Calculate unit vector k

In [17]:
import numpy as np

def unit_vector_k(u, v, w):
    """Determines unit vector k (parallel to new z-axis)
    k: unit vector parallel to new coordinate z axis
    b[0]: instrument offset in w1
    Adapted from Lee, L., W. Massman, and B. Law, 2004: Handbook of Micrometeorology, Chapt 3, Section 9
    """
    su = np.mean(u)
    sv = np.mean(v)
    sw = np.mean(w)
    suv = np.mean(u * v)
    suw = np.mean(u * w)
    svw = np.mean(v * w)
    su2 = np.mean(u * u)
    sv2 = np.mean(v * v)

    h = np.array([
        [1, su, sv],
        [su, su2, suv],
        [sv, suv, sv2]
    ])
    g = np.array([sw, suw, svw])

    b = np.linalg.lstsq(h, g, rcond=None)[0]

    # Determine unit vector k
    unit_k = np.zeros(b.size)
    unit_k[2] = 1.0 / (1.0 + b[1] * b[1] + b[2] * b[2])
    unit_k[0] = -b[1] * unit_k[2]
    unit_k[1] = -b[2] * unit_k[2]
    unit_k /= np.linalg.norm(unit_k)

    print('\nUnit vector k of planar fit coordinate:')
    print(f'  k_vector = [{unit_k[0]:.3f}, {unit_k[1]:.3f}, {unit_k[2]:.3f}]')
    print(f'  b0 = {b[0]:.3f}')

    return unit_k, b[0]

# Calculate 30-min flux

In [18]:
import pandas as pd
import numpy as np
import math

def calculate_fluxes(unit_k, pressure_kpa, df):
    # Transformation to the natural wind coordinate system
    # (Lee, L., W. Massman, and B. Law, 2004: Handbook of Micrometeorology, Chapt 4)

    u_bar = np.mean(df['Ux'])
    v_bar = np.mean(df['Uy'])
    w_bar = np.mean(df['Uz'])

    # Calculate wind direction and speed
    ce = u_bar / math.sqrt(u_bar * u_bar + v_bar * v_bar)
    se = v_bar / math.sqrt(u_bar * u_bar + v_bar * v_bar)

    eta = math.acos(ce) / math.pi * 180.0
    eta = 360.0 - eta if se < 0 else eta
    eta = (360.0 - eta + CSAT3_AZIMUTH) % 360.0
    print(f'  Wind direction: {eta:.3f} degree')

    # Calculate unit vector i and j
    unit_j = np.cross(unit_k, [u_bar, v_bar, w_bar])
    unit_j /= np.linalg.norm(unit_j)
    unit_i = np.cross(unit_j, unit_k)

    u = df['Ux_fluct'] * unit_i[0] + df['Uy_fluct'] * unit_i[1] + df['Uz_fluct'] * unit_i[2]
    v = df['Ux_fluct'] * unit_j[0] + df['Uy_fluct'] * unit_j[1] + df['Uz_fluct'] * unit_j[2]
    w = df['Ux_fluct'] * unit_k[0] + df['Uy_fluct'] * unit_k[1] + df['Uz_fluct'] * unit_k[2]

    ust = math.sqrt(math.sqrt(np.mean(u * w) * np.mean(u * w) + np.mean(v * w) * np.mean(v * w)))
    print(f'  Friction velocity: {ust:.3f} m/s')

    rho = 1.20

    f0 = np.mean(w * df['co2_fluct'])
    e0 = np.mean(w * df['h2o_fluct']) / 1000.0
    sh0 = rho * C_AIR * np.mean(w * df['Ts_fluct'])

    if pressure_kpa is not None:
        p = pressure_kpa * 1000.0
        ta = np.mean(df['Ts']) + 273.15
        rho_v = np.mean(df['h2o']) / 1000.0
        rho_c = np.mean(df['co2'])

        vp = rho_v * RD * ta / 0.622
        q = 0.622 * vp / (p - 0.378 * vp)
        rho = p / (RD * (1.0 + 0.608 * q) * ta)
        rho_d = rho - rho_v

        sh = rho * C_AIR * np.mean(w * df['Ts_fluct'])

        # Sonic correction below is commented out because it is already done in LI-7500
        # H = H + rho * C_AIR * (-0.51 * ta * np.mean(w * df['h2o_fluct'] / 1000.0) / rho

        e = (1.0 + 1.6077 * rho_v / rho_d) * (e0 + sh / rho / C_AIR * rho_v / ta)
        fc = f0 + 1.6077 * e / rho_d * rho_c / (1.0 + 1.6077 * (rho_v / rho_d)) + sh / rho / C_AIR * rho_c / ta

        print(f'  Air pressure = {p:.2f} Pa')
        print(f'  Air density = {rho:.2f} kg/m3')
        print(f'  Sensible heat flux = {sh:.2f} W/m2')
        print(f'  H2O flux before WPL correction = {e0 * LV:.2f} kg/m2')
        print(f'  H2O flux after WPL correction = {e * LV:.2f} kg/m2')
        print(f'  CO2 flux before WPL correction = {f0 * 1000.0 / 44.0:.2f} umol/m2/s')
        print(f'  CO2 flux after WPL correction = {fc * 1000.0 / 44.0:.2f} umol/m2/s')
    else:
        print(f'  Sensible heat flux: {sh0:.2f} umol/m2/s')
        print(f'  Latent heat flux: {e0 * LV:.2f} W/m2')
        print(f'  CO2 flux: {f0 * 1000.0 / 44.0:.2f} W/m2')

# Get pressure

In [19]:
def get_pressure(start, end, df):
    """Get average air pressure for the averaging time period
    """
    sub_df = df[(df['TmStamp'] >= start) & (df['TmStamp'] < end)]

    if len(sub_df) == 0:
        print('  No pressure data available')
        return None

    return sub_df['pressure_irga_mean'].mean()

# Main program

In [20]:
import numpy as np
import pandas as pd
import argparse
from setting import AVERAGING_PERIOD_MINUTES
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

files = ['../data/test.txt']
start_of_month = datetime(2023, 9, 1)
end_of_month = start_of_month + relativedelta(months=1)
pres_file = '../data/202309_ten.txt'

# Read all files
df = pd.DataFrame()
print(f'Reading {len(files)} {"files" if len(files) > 1 else "file"}:')
for f in files:
    print(f'  {f}')
    _df = pd.read_csv(f)
    df = pd.concat([df, _df], ignore_index=True)
df['TmStamp'] = pd.to_datetime(df['TmStamp'])

if pres_file:
    WPL = True
    pres_df = pd.read_csv(pres_file)
    pres_df['TmStamp'] = pd.to_datetime(pres_df['TmStamp'])
else:
    pres_df = pd.DataFrame()
    WPL = False

# Data file Diag configuration
#
# 11 | 10 |  9 |  8 |  7 |  6 |  5 |  4 |  3 |  2 |  1 |  0 |
#    CSAT3 flags    |    IRGA flags     |    AGC/6.25       |
# CSAT3:
# 9: lost trigger special case
# 10: no data special case
# 11: wrong CSAT3 embedded code special case
# 12: SDM error special case
# 13: NaN special case
#
# IRGA:
# 1000: chopper
# 0100: detector
# 0010: pll
# 0001: sync

# Determine unit vector k of planar fit coordinate
# (Lee, L., W. Massman, and B. Law, 2004: Handbook of Micrometeorology, Chapt 3, Section 3)
# unit_k is unit vector parallel to new coordinate z axis
# bo is instrument offset in w1
df = df[df['diag'] < 256]  # Remove bad CSAT3 data
unit_k, b0 = unit_vector_k(df['Ux'], df['Uy'], df['Uz'])

periods = pd.date_range(start_of_month,end_of_month,freq=f'{AVERAGING_PERIOD_MINUTES}min').to_list()
for i in range(len(periods) - 1):
    start = periods[i]
    end = periods[i + 1]
    sub_df = df[(df['TmStamp'] >= start) & (df['TmStamp'] < end) & (df['diag'] < 16)]
    if len(sub_df) == 0:
        continue

    print(f'\n{end.strftime("%Y-%m-%d %H:%M")} {len(sub_df)}')

    flags = {}
    quality_control(unit_k, sub_df, flags)

    pressure_kpa = get_pressure(start, end, pres_df) if WPL else np.nan
    calculate_fluxes(unit_k, pressure_kpa, sub_df)

Reading 1 file:
  ../data/test.txt

Unit vector k of planar fit coordinate:
  k_vector = [-0.141, 0.053, 0.989]
  b0 = 0.106

2023-09-01 00:30 6000
  Available data fraction: 0.333

  Diagnostics                                     Ux                  Uy                  Uz                  Ts                 co2                 h2o
  Spike fraction                               0.000               0.000               0.001               0.000               0.000               0.000
  Empty bin ratio                              0.200               0.210               0.260               0.230               0.210               0.290
  Dropouts, extreme dropouts            0.001, 0.000        0.001, 0.000        0.001, 0.001        0.009, 0.009        0.009, 0.009        0.017, 0.017
  Range                             [-1.669, -0.567]     [-1.140, 0.103]     [-0.328, 0.382]      [20.00, 21.80]       [9.38, 10.86]    [690.32, 711.72]
  Skewness, kurtosis                   -0.097, 2.481 